In [12]:
from importlib import reload
from qt_app import app,eastmoney_parser,ratio_calculator,utils,kelly
utils = reload(utils)
kelly = reload(kelly)

import nest_asyncio
nest_asyncio.apply()

config = utils.config
data_path = config['data_path']
pkg_path = utils.pkg_path

In [13]:
import asyncio
from pyppeteer import launch
import pandas as pd
from bs4 import BeautifulSoup
import csv


async def main(end_page):
    # 启动浏览器
    browser = await launch()
    page = await browser.newPage()
    url = "https://finance.yahoo.com/research" #Accessable only with US IP
    "https://finance.yahoo.com/research?investment_rating=Bullish"
    await page.goto(url)
    print('打开网站完成')

    for pagenumber in range(1, end_page+1):
        
        print(f'开始翻页，现在是第{pagenumber}页')
        await asyncio.sleep(10)
        if pagenumber != 1:
            button_xpath = "//button[span/span[text()='Next']]"
            next_button = await page.waitForXPath(button_xpath, timeout=10000)  # 等待按钮出现
            await next_button.click()  # 点击按钮
            
        else: asyncio.sleep(3)
        
        
        print(f'翻到第{pagenumber}页完成')
        await asyncio.sleep(10)
        
        # Extract the content of the page
        content = await page.content()
        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(content, 'html.parser')
        # Lists to hold the extracted data
        data_list = []
        # Extract table rows
        rows = soup.find_all("tr")
        # Iterate over the rows and extract required data
        for row in rows[1:]:  # Skipping the header row
            columns = row.find_all("td")
            if len(columns) > 4:  # To ensure we have the required columns
                report_data = columns[0].text
                # Extracting rating and price target
                rating = None
                price_target = None
                if "Rating:" in report_data:
                    rating_data = report_data.split("Rating:")[1].split("Price Target:")
                    rating = rating_data[0].strip().split('\n')[0] if len(rating_data) > 0 else None
                    price_target = rating_data[1].strip() if len(rating_data) > 1 else None
                
                report_name = report_data.split("Rating:")[0].strip()
                symbols = columns[1].text.strip()
                sector = columns[2].text.strip()
                provider = columns[3].text.strip()
                date = columns[4].text.strip()
                data_list.append([report_name, symbols, sector, provider, date, rating, price_target])
        
        print('数据读取完成')

        # 保存到Excel
        # File path for the CSV
        csv_path = f"../data/eastmoney_parser/grpStockReportSummary/yahoofinance_{pagenumber}.csv"
        # Write the extracted data to the CSV file
        with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            # Write the headers
            writer.writerow(['report_name', 'symbols', 'sector', 'provider', 'date', 'rating', 'price_target'])
            # Write the extracted data
            for row in data_list:
                writer.writerow(row)

        print(f'保存为yahoofinance_{pagenumber}.xlsx完成') #'''
        

    await browser.close()
    '''
    # 打开并合并表格
    path = "F:/股票研报评级/"
    all_files = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

    all_dfs = []
    for file in sorted(all_files):
        df = pd.read_excel(file)
        df = df.drop(df.index[0])  # 删除第一行
        all_dfs.append(df)

    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_excel(os.path.join(path, "stock.xlsx"), index=False)
    print('所有表格合并完成')#'''

asyncio.get_event_loop().run_until_complete(main(end_page=2)) 


打开网站完成
开始翻页，现在是第1页
翻到第1页完成
数据读取完成
[["Morningstar | A Weekly Summary of Stock Ideas and Developments in the Companies We CoverIn this edition, Markel is no mini-Berkshire; Cigna eyes bigger slice of PBM pie; Gap's struggles; and Guidewire Software, WestRock, and Asbury Automotive Group.", 'LULU,WRK,GWRE,ABG,CI,BRK-A,GPS,MKL', 'Financial Services,Technology,Healthcare,Consumer Cyclical', 'Morningstar', 'Sep 09, 2023', None, None]]
保存为yahoofinance_1.xlsx完成
开始翻页，现在是第2页
翻到第2页完成
数据读取完成
[["Morningstar | A Weekly Summary of Stock Ideas and Developments in the Companies We CoverIn this edition, Markel is no mini-Berkshire; Cigna eyes bigger slice of PBM pie; Gap's struggles; and Guidewire Software, WestRock, and Asbury Automotive Group.", 'LULU,WRK,GWRE,ABG,CI,BRK-A,GPS,MKL', 'Financial Services,Technology,Healthcare,Consumer Cyclical', 'Morningstar', 'Sep 09, 2023', None, None]]
保存为yahoofinance_2.xlsx完成


In [3]:


# Load the HTML content
with open("/mnt/data/Research Reports & Trade Ideas - Yahoo Finance.html", "r", encoding="utf-8") as file:
    content = file.read()

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

# Lists to hold the extracted data
data_list = []

# Extract table rows
rows = soup.find_all("tr")

# Iterate over the rows and extract required data
for row in rows[1:]:  # Skipping the header row
    columns = row.find_all("td")
    
    if len(columns) > 4:  # To ensure we have the required columns
        report_data = columns[0].text
        
        # Extracting rating and price target
        rating = None
        price_target = None
        
        if "Rating:" in report_data:
            rating_data = report_data.split("Rating:")[1].split("Price Target:")
            rating = rating_data[0].strip().split('\n')[0] if len(rating_data) > 0 else None
            price_target = rating_data[1].strip() if len(rating_data) > 1 else None
        
        report_name = report_data.split("Rating:")[0].strip()
        symbols = columns[1].text.strip()
        sector = columns[2].text.strip()
        provider = columns[3].text.strip()
        date = columns[4].text.strip()

        data_list.append([report_name, symbols, sector, provider, date, rating, price_target])

# Checking the first few records again
data_list[:5]

csv_filepath:  ../data/short_term_solvency_ratio\tblStockPickedWithSolvencyR2023-08-23.csv
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,603096.SS,600276.SS,688078.SS,601398.SS,000848.SZ
Date,,,,,
2023-07-14,20.500000,47.520000,50.000000,4.5665,8.80
2023-07-17,20.389999,47.200001,50.349998,4.5700,8.73
2023-07-18,20.070000,47.200001,49.570000,4.5400,8.72
2023-07-19,20.530001,46.689999,50.009998,4.5600,8.73
2023-07-20,20.129999,45.959999,48.849998,4.5800,8.74
2023-07-21,20.129999,45.930000,47.750000,4.5800,8.91
2023-07-24,20.250000,47.290001,48.650002,4.5700,8.91
2023-07-25,20.450001,47.759998,48.770000,4.6200,8.96
2023-07-26,20.260000,48.080002,47.480000,4.6300,9.01
